# Data Preprocessing
This notebook parses and processes the API responses from the "PlacementSuggestionService".

In [1]:
import os
import json
import glob

from tqdm import tqdm
import pandas as pd

In [2]:
# input
data_in = '../data/input/placements_api/'
fn_hate_info = '../data/input/hate_terms_additional_info.csv'

# outputs
data_dir = '../data/output/placements_api_keyword_status/'
os.makedirs(data_dir, exist_ok=True)
fn_hate = os.path.join(data_dir, "hate.csv")
fn_social_justice = os.path.join(data_dir, "social_justice.csv")
fn_policy = os.path.join(data_dir, "policy.csv")
fn_basewords = os.path.join(data_dir, "basewords.csv")
fn_adhoc = os.path.join(data_dir, "adhoc.csv")

data_dir_2 = '../data/output/placements_api_suggestions/'
os.makedirs(data_dir_2, exist_ok=True)
fn_social_justice_videos = os.path.join(data_dir_2, 'videos_for_social_justice_terms.csv')
fn_hate_videos = os.path.join(data_dir_2, 'videos_for_hate_terms.csv')
fn_social_justice_channels = os.path.join(data_dir_2, 'channels_for_social_justice_terms.csv')
fn_hate_channels = os.path.join(data_dir_2, 'channels_for_hate_terms.csv')

In [3]:
files = glob.glob(data_in + '*/*.json')
len(files)

802

## Checking the API responses
Refer to `../data/reference/placements_api_example_responses/full.json` for an example what a typical API response looks like.

In [4]:
errors = {}
dataset = []
for fn in tqdm(files):
    record = {
        'fn' : fn,
        'search_term' : fn.replace(data_in, '').split('/')[-1].replace('+', ' ').replace('.json', '')
    }
    data = json.load(open(fn))
    if data == dict():
        record['is_blocked'] = True
        dataset.append(record)
        continue
    else:
        record['is_blocked'] = False
    try:
        # 4 - YouTube channel suggestions
        youtube_channels_ = data.get('4')
        if youtube_channels_:
            youtube_channels_number = youtube_channels_['2']
            youtube_channels = youtube_channels_.get('1', [])

            channel_meta = []
            for youtube_channel in youtube_channels:
                youtube_channel_meta_ = youtube_channel['8']

                row = dict(
                    youtube_channel_id = youtube_channel['7']['1']['1'],
                    youtube_channel_name = youtube_channel['7']['2'],
                    youtube_channel_subs = youtube_channel_meta_.get('2'),
                    youtube_channel_videos = youtube_channel_meta_['1'],
                    youtube_channel_thumbnail = youtube_channel_meta_['3']
                )
                channel_meta.append(row)
            record['n_youtube_channels'] = youtube_channels_number
            record['youtube_channels'] = channel_meta
    except:
        pass
    try:
        # 5 - YouTube video suggestions
        youtube = data.get('5')
        if youtube:
            youtube_number_videos = youtube['2']
            youtube_videos = youtube.get('1', [])
            
            youtube_video_meta = []
            for youtube_video in youtube_videos:
                row = dict(
                    youtube_video_id = youtube_video['1'],
                    youtube_video_title = youtube_video['2'],
                    youtube_video_views = youtube_video['3'],
                    youtube_video_channel = youtube_video['4']    
                )
                youtube_video_meta.append(row)
            record['n_youtube_videos'] = youtube_number_videos
            record['youtube_videos'] = youtube_video_meta
    except:pass
    dataset.append(record)

100%|██████████| 802/802 [00:00<00:00, 14556.31it/s]


In [5]:
df = pd.DataFrame(dataset)

In [6]:
for col in ['n_youtube_channels', 'n_youtube_videos']:
    df[col] = df[col].astype(float)

In [7]:
def determine_status(row: dict) -> str:
    """
    Determines the status (`full`, `blocked`, `partially blocked` or `empty`) 
    of or a given keyword's response.
    
    Examples for each kind of response in 
    `../data/reference/placements_api_response_examples`. 
    
    Please read the methodology for more detail.
    """
    if row['is_blocked'] == True:
        return 'Blocked'
    elif row['n_youtube_channels'] == 0 and row['n_youtube_videos'] == 0:
        return 'Empty'
    elif row['n_youtube_videos'] == 1:
        return 'Partial Block'
    return 'Full'

In [8]:
df['status'] = df.apply(determine_status, axis=1)

In [9]:
blocked = df[df.fn.str.contains('/blocked/')]

In [10]:
merge_cols = [
    'search_term', 
    'is_blocked', 
    'status', 
    'n_youtube_videos', 
    'n_youtube_channels'
]

df = df.merge(blocked[merge_cols], 
              on=['search_term'], how='left', 
              suffixes=('', '_no_spaces'))

In [11]:
# create order of statuses for sorting
status_order = ["Full", "Empty", "Partial Block", "Blocked"]
for col in ['status', 'status_no_spaces']:
    df[col] = pd.Categorical(df[col], status_order)

In [12]:
df.sort_values(by=['status',
                   'search_term',
                   'status_no_spaces'], 
               ascending=True, 
               inplace=True)

In [13]:
hate = df[df.fn.str.contains('/hate/')]
social_justice = df[df.fn.str.contains('/social_justice/')]
policy = df[df.fn.str.contains('/policy/')]
word = df[df.fn.str.contains('/blocked_basewords/')]
adhoc = df[(df.fn.str.contains('adhoc/'))]

In [14]:
len(hate), len(social_justice), len(policy)

(87, 62, 150)

Save the terms and responses:

In [15]:
display_cols = [
    'search_term', 
    'status',
    'status_no_spaces',
    'n_youtube_videos',
    'n_youtube_channels',
    'n_youtube_videos_no_spaces',
    'n_youtube_channels_no_spaces',
]

In [16]:
# add links for additional info on each hate term.
hate = hate.merge(pd.read_csv(fn_hate_info), on='search_term')

# save the status of terms from each keyword list
hate[display_cols + ['additional_info_link']].to_csv(fn_hate, index=False)
social_justice[display_cols].to_csv(fn_social_justice, index=False)
policy[display_cols].to_csv(fn_policy, index=False)
word[display_cols].to_csv(fn_basewords, index=False)
adhoc[display_cols].to_csv(fn_adhoc, index=False)


# Here we write everything to a spreadsheet...
writer = pd.ExcelWriter('../data/reference/what_is_blocked.xlsx')
hate[display_cols + ['additional_info_link']].to_excel(writer, 'hate', index=False)
social_justice[display_cols].to_excel(writer, 'social_justice', index=False)
policy[display_cols].to_excel(writer, 'policy', index=False)
adhoc[display_cols].to_excel(writer, 'adhoc', index=False)
word[display_cols].to_excel(writer, 'basewords', index=False)
writer.save()

In [17]:
df[df['search_term'] == 'black lives matter']

,fn,search_term,is_blocked,n_youtube_channels,youtube_channels,n_youtube_videos,youtube_videos,status,is_blocked_no_spaces,status_no_spaces,n_youtube_videos_no_spaces,n_youtube_channels_no_spaces
760,../data/input/placements_api/blocked/black liv...,black lives matter,False,96118.0,[{'youtube_channel_id': 'UCIveFvW-ARp_B_Rckhwe...,2029797.0,"[{'youtube_video_id': 'A2o15RCtSS0', 'youtube_...",Full,False,Full,2029797.0,96118.0
302,../data/input/placements_api/social_justice/bl...,black lives matter,True,NaN,NaN,NaN,NaN,Blocked,False,Full,2029797.0,96118.0


## What YouTube channels and videos are suggested?

In [18]:
youtube_channels = []
youtube_videos = []
websites = []
for results in [_ for _ in dataset if _['fn'] in df['fn'].tolist()]:
    _row = {'search_term' : results['search_term']}
    
    if results.get('youtube_channels'):
        for channel_meta in results.get('youtube_channels'):
            row = _row.copy()
            row = {**_row, **channel_meta}
            row['channel_url'] = ('youtube.com/channel/'
                                 f'{channel_meta["youtube_channel_id"]}')
            youtube_channels.append(row)
    else:
        youtube_channels.append(_row)
            
    if results.get('youtube_videos'):
        for video_meta in results.get('youtube_videos'):
            row = _row.copy()
            row = {**_row, **video_meta} 
            row['video_url']= ('youtube.com/watch/?v='
                            f'{video_meta["youtube_video_id"]}')
            youtube_videos.append(row)
    else:
        youtube_videos.append(_row)
            
    if results.get('website'):
        for web_meta in results.get('website'):
            row = _row.copy()
            row = {**_row, **web_meta}
            websites.append(row)
    else:
        websites.append(_row)    

In [19]:
df_channel = pd.DataFrame(youtube_channels)
df_vids = pd.DataFrame(youtube_videos)

In [20]:
df_vids = df_vids[~df_vids.youtube_video_id.isnull()]
len(df_vids[df_vids.search_term.isin(hate.search_term)])

1385

In [21]:
# get video id
df_vids["video_id"] = df_vids[~df_vids.video_url.isnull()].apply(
    lambda x: x['video_url'].split('?v=')[-1] if x['video_url'] else None, 
    axis=1
)

In [22]:
def get_bandaid_search_term(search_term):
    """Make clear if the search_term was a space-removal term"""
    if search_term in df[~df.status_no_spaces.isnull()].search_term.tolist():
        return search_term.replace(' ', '')
    return search_term

In [23]:
df_vids.loc[:, "search_term"] = df_vids.search_term.apply(get_bandaid_search_term)
df_channel.loc[:, "search_term"] = df_channel.search_term.apply(get_bandaid_search_term)

In [24]:
# how many unique videos were suggested from the hate terms?
df_vids[df_vids.search_term.isin(hate.search_term)].youtube_video_id.nunique()

1140

In [25]:
# save the parsed suggested videos
df_vids[df_vids.search_term.isin(hate.search_term)].to_csv(
    fn_hate_videos, index=False
)
df_vids[df_vids.search_term.isin(social_justice.search_term)].to_csv(
    fn_social_justice_videos, index=False
)

In [26]:
# save the parsed suggested channels
df_channel[df_channel.search_term.isin(hate.search_term)].to_csv(
    fn_hate_channels, index=False
)
df_channel[df_channel.search_term.isin(social_justice.search_term)].to_csv(
    fn_social_justice_channels, index=False
)